In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import optuna
from optuna.integration import TFKerasPruningCallback
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import cv2

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [27]:
# Loading data
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150

def get_training_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img_arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img_arr is None:
                    continue
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
    return np.array(data, dtype=object)

In [28]:
# Load data
train = get_training_data(r'C:\Users\danie\Leiden_Scripts\Datascience2025_A2\data\chest_xray\train')
test = get_training_data(r'C:\Users\danie\Leiden_Scripts\Datascience2025_A2\data\chest_xray\test')
val = get_training_data(r'C:\Users\danie\Leiden_Scripts\Datascience2025_A2\data\chest_xray\val')

# Prepare data for training
X_train = np.array([i[0] for i in train]).reshape(-1, img_size, img_size, 1)
y_train = np.array([i[1] for i in train])
X_test = np.array([i[0] for i in test]).reshape(-1, img_size, img_size, 1)
y_test = np.array([i[1] for i in test])
X_val = np.array([i[0] for i in val]).reshape(-1, img_size, img_size, 1)
y_val = np.array([i[1] for i in val])

In [29]:
def create_model(trial):
    """Create a CNN model with hyperparameters to be optimized"""
    # Hyperparameters to optimize
    n_layers = trial.suggest_int('n_layers', 2, 5)
    initial_filters = trial.suggest_int('initial_filters', 16, 64)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    
    model = keras.Sequential()
    
    # First layer
    model.add(layers.Conv2D(initial_filters, (3, 3), activation='relu', input_shape=(150, 150, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Additional layers
    for i in range(n_layers - 1):
        filters = initial_filters * (2 ** (i + 1))
        model.add(layers.Conv2D(filters, (3, 3), activation='relu', padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    return model, batch_size

In [31]:
def objective(trial):
    """Objective function for Optuna optimization"""
    # Create model
    model, batch_size = create_model(trial)
    
    # Data augmentation
    datagen = ImageDataGenerator(
        rotation_range=trial.suggest_int('rotation_range', 0, 30),
        width_shift_range=trial.suggest_float('width_shift_range', 0, 0.2),
        height_shift_range=trial.suggest_float('height_shift_range', 0, 0.2),
        horizontal_flip=trial.suggest_categorical('horizontal_flip', [True, False]),
        fill_mode='nearest'
    )
    
    # Training
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        epochs=10,
        validation_data=(X_val, y_val),
        callbacks=[
            TFKerasPruningCallback(trial, 'val_accuracy'),
            keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
        ]
    )
    
    return history.history['val_accuracy'][-1]

In [32]:
# Create study and optimize
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=50)

[I 2025-04-11 16:22:01,379] A new study created in memory with name: no-name-e64fd236-ecd3-44a1-885c-cd94052f2fec


Epoch 1/10


c:\Users\danie\Leiden_Scripts\Datascience2025_A2\datascience_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


326/326 ━━━━━━━━━━━━━━━━━━━━ 100s 298ms/step - accuracy: 0.8338 - loss: 0.6196 - val_accuracy: 0.8125 - val_loss: 0.6760
Epoch 2/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 104s 318ms/step - accuracy: 0.9038 - loss: 0.2320 - val_accuracy: 0.8750 - val_loss: 0.7124
Epoch 3/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 97s 297ms/step - accuracy: 0.9345 - loss: 0.1701 - val_accuracy: 0.6250 - val_loss: 2.0475
Epoch 4/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 96s 295ms/step - accuracy: 0.9364 - loss: 0.1668 - val_accuracy: 0.5000 - val_loss: 3.0310
Epoch 5/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 102s 313ms/step - accuracy: 0.9396 - loss: 0.1413 - val_accuracy: 0.5625 - val_loss: 1.2608


[I 2025-04-11 16:30:20,795] Trial 0 finished with value: 0.5625 and parameters: {'n_layers': 4, 'initial_filters': 35, 'dropout_rate': 0.27252680243361316, 'learning_rate': 0.000269921204261943, 'batch_size': 16, 'rotation_range': 13, 'width_shift_range': 0.14282978902482193, 'height_shift_range': 0.07900316538334176, 'horizontal_flip': False}. Best is trial 0 with value: 0.5625.


...

Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 124s 744ms/step - accuracy: 0.8202 - loss: 2.2003 - val_accuracy: 0.8125 - val_loss: 0.3576
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 121s 742ms/step - accuracy: 0.9153 - loss: 0.2076 - val_accuracy: 0.6250 - val_loss: 0.4429
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 120s 737ms/step - accuracy: 0.9221 - loss: 0.2642 - val_accuracy: 0.8750 - val_loss: 0.4068
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 120s 738ms/step - accuracy: 0.9310 - loss: 0.1893 - val_accuracy: 0.8125 - val_loss: 0.4287
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 120s 739ms/step - accuracy: 0.9402 - loss: 0.1529 - val_accuracy: 0.6875 - val_loss: 1.1785
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 122s 749ms/step - accuracy: 0.9547 - loss: 0.1290 - val_accuracy: 0.9375 - val_loss: 0.2792
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 124s 759ms/step - accuracy: 0.9475 - loss: 0.1308 - val_accuracy: 0.7500 - val_loss: 0.5288
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 130s 795ms/step - accuracy: 0.9517 -

[I 2025-04-11 17:32:39,665] Trial 14 finished with value: 0.625 and parameters: {'n_layers': 3, 'initial_filters': 48, 'dropout_rate': 0.18910105692981835, 'learning_rate': 0.0011798620650160124, 'batch_size': 32, 'rotation_range': 18, 'width_shift_range': 0.023417003418554717, 'height_shift_range': 0.09194975759807503, 'horizontal_flip': False}. Best is trial 6 with value: 0.875.


Epoch 1/10
133/326 ━━━━━━━━━━━━━━━━━━━━ 33s 176ms/step - accuracy: 0.7939 - loss: 0.8635

[W 2025-04-11 17:33:06,678] Trial 15 failed with parameters: {'n_layers': 5, 'initial_filters': 24, 'dropout_rate': 0.32779288863964207, 'learning_rate': 0.00037563237635704187, 'batch_size': 16, 'rotation_range': 24, 'width_shift_range': 0.06449078095022842, 'height_shift_range': 0.05039862156484426, 'horizontal_flip': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\danie\Leiden_Scripts\Datascience2025_A2\datascience_env\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_12080\3338140986.py", line 16, in objective
    history = model.fit(
              ^^^^^^^^^^
  File "c:\Users\danie\Leiden_Scripts\Datascience2025_A2\datascience_env\Lib\site-packages\keras\src\utils\traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "c

KeyboardInterrupt: 

In [4]:
# Print best parameters
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:


NameError: name 'study' is not defined

In [1]:
# Visualize optimization history
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_param_importances(study)
optuna.visualization.plot_parallel_coordinate(study)

NameError: name 'optuna' is not defined

In [ ]:
# Train final model with best parameters
best_params = study.best_params
model, batch_size = create_model(study.best_trial)

# Load all data
(x_train, y_train), (x_val, y_val), (x_test, y_test) = get_training_data()

# Data augmentation with best parameters
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=best_params['rotation_range'],
    width_shift_range=best_params['width_shift_range'],
    height_shift_range=best_params['height_shift_range'],
    horizontal_flip=best_params['horizontal_flip'],
    fill_mode='nearest'
)

# Train final model
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    epochs=20,
    validation_data=(x_val, y_val),
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5),
        keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=2, factor=0.3)
    ]
)

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy*100:.2f}%")

In [ ]:
# Import additional metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import seaborn as sns

# Function to evaluate and plot metrics
def evaluate_model(model, x_data, y_data, title_prefix):
    # Get predictions
    y_pred = (model.predict(x_data) > 0.5).astype(int)
    
    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y_data, y_pred, average='binary')
    
    # Create confusion matrix
    cm = confusion_matrix(y_data, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{title_prefix} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    # Plot metrics
    plt.subplot(1, 2, 2)
    metrics = ['Precision', 'Recall', 'F1 Score']
    values = [precision, recall, f1]
    plt.bar(metrics, values)
    plt.title(f'{title_prefix} Metrics')
    plt.ylim(0, 1)
    for i, v in enumerate(values):
        plt.text(i, v + 0.02, f'{v:.3f}', ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed classification report
    print(f"\n{title_prefix} Classification Report:")
    print(classification_report(y_data, y_pred))
    
    return precision, recall, f1

# Evaluate CNN model on train and test sets
print("\nEvaluating CNN Model:")
train_metrics = evaluate_model(model, x_train, y_train, "Train")
test_metrics = evaluate_model(model, x_test, y_test, "Test")

# Load and evaluate the alternative ML model (Random Forest)
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load the saved Random Forest model
rf_model = joblib.load('../models/random_forest_model.joblib')

# Reshape data for Random Forest (flatten images)
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Evaluate Random Forest model
print("\nEvaluating Random Forest Model:")
rf_train_metrics = evaluate_model(rf_model, x_train_flat, y_train, "Train")
rf_test_metrics = evaluate_model(rf_model, x_test_flat, y_test, "Test")

# Compare models
print("\nModel Comparison Summary:")
print("CNN Model:")
print(f"Train - Precision: {train_metrics[0]:.3f}, Recall: {train_metrics[1]:.3f}, F1: {train_metrics[2]:.3f}")
print(f"Test  - Precision: {test_metrics[0]:.3f}, Recall: {test_metrics[1]:.3f}, F1: {test_metrics[2]:.3f}")
print("\nRandom Forest Model:")
print(f"Train - Precision: {rf_train_metrics[0]:.3f}, Recall: {rf_train_metrics[1]:.3f}, F1: {rf_train_metrics[2]:.3f}")
print(f"Test  - Precision: {rf_test_metrics[0]:.3f}, Recall: {rf_test_metrics[1]:.3f}, F1: {rf_test_metrics[2]:.3f}")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()